In [ ]:
#!sudo apt-get install p7zip-full

In [ ]:
#!7z x /content/raw_data.csv.7z.001

In [ ]:
import pandas as pd
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

#dir = "/content/"
dir = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/Processed/Dataframe/"

df = pd.read_csv(dir+"raw_data.csv")
df['entsFundamentacao'] = df['entsFundamentacao'].apply(eval)

In [2]:
df

,sentenca,reclamante,reclamado,processo,vara,juiz,relatorio,fundamentacao,dispositivo,classificacao,entsFundamentacao
0,PODER JUDICIÁRIO JUSTIÇA DO TRABALHO TRIBUNA...,DANIEL DA SILVA SOUZA DE ALMEIDA,"ESPACO CIDADANIA E OPORTUNIDADES SOCIAIS, UNI...",0101099-10.2020.5.01.0341,12,THIAGO RABELO DA COSTA Juiz do Trabalho Subs...,Trata-se de Reclamação Trabalhista ajuizada ...,. DA INCOMPETÊNCIA MATERIAL NA JUSTIÇA DO TRA...,"Isto posto, declaro a responsabilidade subsidi...",Parcialmente procedente,"[RR, 83, 350, 0010, 2009, 2018, 0000, 577900, ..."
1,Relatório PODER JUDICIÁRIO FEDERAL JUSTIÇA DO...,LAILTON DE CARVALHO MOURA,WM TRANSPORTE DE CARGAS E PASSAGEIROS LIDA M...,0100540-28.2017.5.01.0060,"602 do Rio de Janeiro RUA DO LAVRADIO, 132...","ag RIO DE JANEIRO, 13 de Agosto de 2018 ROBE...",PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO...,DA DECLARAÇÃO DA REVELIA DA 1º RECLAMADA E D...,"Posto isto, este magistrado, no exercício da...",Procedente,"[-, CCT, 88., OJSDI, Carta Magna, EC nº 45 / 0..."
2,PODER JUDICIARIO JUSTIÇA DO TRABALHO TRIBUNA...,VERTEYNE RODRIGO MARINS DE SOUZA,"JPTE ENGENHARIA LTDA., PETROLEO BRASILEIRO S ...",01008667520205010482-DEJT-13-02-2021,2º,MARCO ANTONIO MATTOS DE LEMOS Juiz do Trabal...,Vistos etc. VERTEYNE RODRIGO MARINS DE SOUZ...,Das Preliminares. Da Recuperação Judicial A ...,"POSTO ISSO, indefiro, por ora, a gratuidade ...",Parcialmente procedente,"[ADC nº 16, Lei nº 13. 303, art. 105 do NCP, C..."
3,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO T...,ADRIANA PEREIRA SANTANA,AFEQUE SERVICOS DE VIGILANCIA EIRELI e outros,001 1560-11.2015.5.01.0017 |,17º,"RIO DE JANEIRO, 29 de Junho de 2017 CLARISS...","ADRIANA PEREIRA SANTANA, qualificada na inic...",Incompetência absoluta da Justiça do Trabalho...,"Diante do exposto, na forma da fundamentação...",Parcialmente procedente,"[ADC 16, art. 769 da CLT, Lei n. º 8. 666, art..."
4,Relatório Fundamentação Dispositivo PODER J...,,TRANS-EXPERT VIGIILANCIA E TRANSPORTE DE VALOR...,00116209620155010012-DEJT-31-08-2017,12º,ELETÍCIA MARINHO MENDES GOMES DA SILVAJuiza do...,Dispositivo PODER JUDICIÁRIO FEDERAL JUS...,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABA...,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALH...,Parcialmente procedente,"[75, 01., 0012, 0015, 2016, 0001, 0028, 2004, ..."
...,...,...,...,...,...,...,...,...,...,...,...
2582,PODER JUDICIÁRIO JUSTIÇA DO TRABALHO TRIBUNA...,MARIANE MATOS DA PENHA IABAS INSTITUTO DE ...,IABAS INSTITUTO DE ATENCAO BASICA E AVANCADA...,0101103-51.2019.5.01.0060 MARIANE MATOS DA ...,602,CLARISSA SOUZA POLIZELI Juíza do Trabalho Su...,"MARIANE MATOS DA PENHA, qualificada na inici...",Limitação aos valores postulados Conforme a...,"Diante do exposto, na forma da fundamentação...",Parcialmente procedente,"[ADC 16, Lei n. º 8. 666, artigo 2º da CLT, ar..."
2583,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO T...,OLGA PINHEIRO FERREIRA PINTO,BIOTECH HUMANA ORGANIZACAO SOCIAL DE SAUDE e ...,0101720-39.2016.5.01.0020,20º,Cissa de Almeida Biasoli Juíza do Trabalho ...,OLGA PINHEIRO FERREIRA ajuizou ação trabalhi...,Justica Gratuita d Dispõe o art. 99 do CPC/...,"Posto isso, decide esse juízo julgar proceden...",Parcialmente procedente,"[OJ, 382, 400, SDI1 -, 10. do, 06 / 2013, pará..."
2584,Relatório PODER JUDICIÁRIO FEDERAL JUSTIÇA DO ...,,CERVEJARIA PETRÓPOLIS SIA pleiteando a determi...,01003294020205010301-DEJT-25-09-2020,ta de Petrópolis PAP nº: 0100329-40.2020.5....,ROSANGELA KRAUS DE OLIVEIRA MORELI Juíza do ...,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO ...,ISTO POSTO A presente controvérsia está send...,"PELO EXPOSTO,a 1º Vara do Trabalho de Petrópo...",Improcedente,"[art. 381 do CPC, art. 400 do CPC, Justiça do ..."
2585,PODER JUDICIÁRIO FEDERAL JUSTIÇA DO TRABALHO T...,FABRICIO BRANDT MARTINS,EISA PETRO-UM S.A. - EM RECUPERACAO JUDICIAL ...,0100810-91.2017.5.01.0241 |,1º,ANA REGINA FIGUEROA FERREIRA DE BARROS Juíz...,Trata-se de reclamação trabalhista movida po...,DA RECUPERAÇÃO JUDICIAL O deferimento da re...,"Posto isto,

In [3]:
def calc_freq_df(df):
    
    df_procedente = df
    
    list_entsProcedente = list()

    # Cria uma grande lista com todas as Entidades do df:
    for i in range(len(df_procedente)):
        list_entsProcedente = list_entsProcedente + df_procedente['entsFundamentacao'][i]

    # Elimina as Entidades duplicadas entre os registros do df:
    list_entsProcedente = list(set(list_entsProcedente))

    # Cria um df_entsProcedente para calcular a frequencia das ents do set(que é uma list) acima.
    list_zero = list()
    list_zero = [0]*len(list_entsProcedente)

    df_entsProcedente = pd.DataFrame(list(zip(list_entsProcedente, list_zero)), columns =['ents','freq'])

    # Conta a frequencia:
    for i in range(len(df_procedente)):
        listEnt = df_procedente['entsFundamentacao'][i]
        for j in range(len(listEnt)):
            if df_procedente['entsFundamentacao'][i][j] in list_entsProcedente:
                pos = df_entsProcedente.index[(df_entsProcedente['ents'] == df_procedente['entsFundamentacao'][i][j])].tolist()[0]
                df_entsProcedente['freq'][pos] = int(df_entsProcedente['freq'][pos]) + 1
                #df_entsProcedente.loc[:,('freq',pos)] = int(df_entsProcedente.loc[:,('freq',pos)]) + 1

    df_entsProcedente.sort_values(by=['freq'], ascending=False, ignore_index=True, inplace=True)
    
    return (df_entsProcedente)

In [4]:
%%time
df_categ_fund = calc_freq_df(df)
# Estrategias:
# Janela deslizante (100 maiores fequencias) com ou sem eliminacao do termo similar posterior
# Estamos montando um dicionario de entidades da seção Fundamentacao: Analisar as frequencias dos termos de 
#   df_entsProcedente. Talvez itens raros tenham alto custo computacional e possam ser descartados sem grandes
#   prejuizos estatisticos.
# Tem ainda o problema da classificacao de tema/assuntos do documento (pelas classes do MPTD?). Fatiar o DF nas 
# classes e aplicar em cada classe a respectiva Analise de Correspondencia entre os termos e a classificacao da
# procedencia.

CPU times: user 10min 42s, sys: 21.9 ms, total: 10min 42s
Wall time: 10min 42s


In [5]:
df_categ_fund.head(50)

,ents,freq
0,artigo 467 da CLT,646
1,ADC nº 16,614
2,art. 467 da CLT,548
3,ADC 16,540
4,Lei 13. 467 / 2017,495
5,AIRR,458
6,artigos 186 e 927 do Código Civil,411
7,Lei nº 13. 467 / 2017,403
8,Súmula 381 do TST,366
9,artigos 467 e 477 da CLT,358


In [6]:
df_categ_fund.describe()

,freq
count,23956.000000
mean,4.711471
std,16.193432
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,646.000000


In [7]:
df_categ_fund

,ents,freq
0,artigo 467 da CLT,646
1,ADC nº 16,614
2,art. 467 da CLT,548
3,ADC 16,540
4,Lei 13. 467 / 2017,495
...,...,...
23951,Súmula nº 12 do C. TST,1
23952,. 2007. 5. 15. 0083,1
23953,"CRFB, art. / º, XXlle XXIX",1
23954,"artigo 8º, 8 1º, da Lei Complementar nº 95 / 98",1


In [28]:
df_categ_fund.to_csv(dir+"df_categ_fund.csv", index=False)

In [15]:
Total = df_categ_fund['freq'].sum()
Metade = round(Total*50/100)
Metade, Metade*2

(56434, 112868)

In [9]:
soma = 0
k = 0

for i in df_categ_fund.index:
    soma = soma + int(df_categ_fund['freq'][i])
    if soma <= Metade:
        k = i
        
print("Indice do pont de corte do 50% tokens mais frequentes, k = ", k)

k =  1125


In [10]:
Metade

56434

In [11]:
soma = 0 
for i in range(1125):
    soma = soma + int(df_categ_fund['freq'][i])
print(soma)    

56398


In [ ]:
# Logo, para i de 0 a 1125 temos 50% da quantidade de tokens. Devemos tratar esses 1125 primeiros Tokens!

In [12]:
%%time
# Testando um modelo de similaridade em portugues hospedado na Huggingface
# https://huggingface.co/rufimelo/Legal-SBERTimbau-sts-large-v2
#!pip install -U sentence-transformers
# Usage (Sentence-Transformers)

from sentence_transformers import SentenceTransformer
sentences = ["Isto é um exemplo", "Isto é um outro exemplo"]

model = SentenceTransformer('rufimelo/Legal-SBERTimbau-sts-large-v2')
embeddings = model.encode(sentences)
#print(embeddings)

CPU times: user 11.6 s, sys: 2.15 s, total: 13.7 s
Wall time: 11.1 s


In [29]:
%%time
# Usage (HuggingFace Transformers)
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('rufimelo/Legal-SBERTimbau-sts-large-v2')
model = AutoModel.from_pretrained('rufimelo/Legal-SBERTimbau-sts-large-v2')


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



def compute_simil(text1, text2):
    # Sentences we want sentence embeddings for
    #sentences = ['This is an example sentence', 'Each sentence is converted']
    #sentences = ["o amor é contagiante", "o amor é contagigante"]
    sentences = list()
    sentences.append(text1)
    sentences.append(text2)


    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    #print("Sentence embeddings cosine similarity:")
    res = cosine_similarity(sentence_embeddings[0].reshape(1,-1),sentence_embeddings[1].reshape(1,-1))[0][0]
    
    return (res)

CPU times: user 4.47 s, sys: 1.4 s, total: 5.87 s
Wall time: 10.3 s


In [16]:
df_categ_fund_50 = df_categ_fund[0:1125]   # <===
df_categ_fund_50

,ents,freq
0,artigo 467 da CLT,646
1,ADC nº 16,614
2,art. 467 da CLT,548
3,ADC 16,540
4,Lei 13. 467 / 2017,495
...,...,...
1120,0102440,19
1121,"art. 5º, caput, da CF",19
1122,artigo 8 3º do artigo 62 da Constituição,19
1123,art. 114 da Constituição Federal,19


In [30]:
# Tratando, inicialmente, os 1125 primeiros itens (mais frequentes e que representam 50% do total de ocorrencias
# no texto)
z = len(df_categ_fund_50) # Podemos segmentar o DF df_categ_fund[0:1125] em df_categ_fund_50 e colocar o j ate len(df_categ_fund_50)

#for i in tqdm(range(z-1)):
i = 101  # continuando o que foi feito ate i = 100.
while i < (z-1):
    print("i = ",i, "/",z, " = z")
    #for j in range(i+1,z):
    j = i+1
    while j < z:   # Nao pode ser <= pois o array começa em 0.
        #print("j = ",j) # Podemos comentar esta linha!
        if compute_simil(df_categ_fund_50['ents'][i],df_categ_fund_50['ents'][j]) >= 0.9:
            print("compute_simil >= 0.9 para [i:",i,"] = ",df_categ_fund_50['ents'][i]," [j:",j,"] = ",df_categ_fund_50['ents'][j])
            df_categ_fund_50['freq'][i] = df_categ_fund_50['freq'][i] + df_categ_fund_50['freq'][j] # Acumula a frequencia
            #df_categ_fund['ents'][j] = df_categ_fund['ents'][i] # Ou podemos excluir as demais/seguintes entidade equivalentes
            df_categ_fund_50.drop(df_categ_fund_50.index[j], inplace=True) # Exclui o item duplicado
            df_categ_fund_50.reset_index(drop=True, inplace=True) # Atualiza o indice do DF.
            z = len(df_categ_fund_50)
            print("Tam. DF: ", z)
        j = j + 1
    i = i + 1

df_categ_fund_50.sort_values(by=['freq'], ascending=False, ignore_index=True, inplace=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


i =  101 / 797  = z
compute_simil >= 0.9 para [i: 101 ] =  art. 373, Il, do CPC de 2015  [j: 770 ] =  art. 378, Il, do CPC / 2015
Tam. DF:  796
i =  102 / 796  = z
compute_simil >= 0.9 para [i: 102 ] =  artigo 15, da Lei 8. 036 / 90  [j: 664 ] =  art. 15 da Lei 8. 036 / 90
Tam. DF:  795
i =  103 / 795  = z
i =  104 / 795  = z
i =  105 / 795  = z
i =  106 / 795  = z
compute_simil >= 0.9 para [i: 106 ] =  artigo 28 da Lei nº 8. 212 / 91  [j: 205 ] =  art. 28, Lei 8. 212 / 91
Tam. DF:  794
compute_simil >= 0.9 para [i: 106 ] =  artigo 28 da Lei nº 8. 212 / 91  [j: 216 ] =  artigo 28 da Lei 8. 212 / 91
Tam. DF:  793
compute_simil >= 0.9 para [i: 106 ] =  artigo 28 da Lei nº 8. 212 / 91  [j: 248 ] =  artigo 28, S 9º, da Lei nº 8. 212 / 91
Tam. DF:  792
compute_simil >= 0.9 para [i: 106 ] =  artigo 28 da Lei nº 8. 212 / 91  [j: 444 ] =  artigo 28, 8 9º, da Lei nº 8. 212 / 91
Tam. DF:  791
compute_simil >= 0.9 para [i: 106 ] =  artigo 28 da Lei nº 8. 212 / 91  [j: 694 ] =  art. 28 da Lei 8. 2

i =  190 / 744  = z
i =  191 / 744  = z
i =  192 / 744  = z
compute_simil >= 0.9 para [i: 192 ] =  art. 14 do CPC  [j: 269 ] =  artigo 14 do CPC
Tam. DF:  743
compute_simil >= 0.9 para [i: 192 ] =  art. 14 do CPC  [j: 715 ] =  art. 14 do CPC / 2015
Tam. DF:  742
i =  193 / 742  = z
compute_simil >= 0.9 para [i: 193 ] =  arts. 159 e 160, |, in fine, CCB / 1916  [j: 666 ] =  aris. 159 e 160, |, in fine, CCB / 1916
Tam. DF:  741
i =  194 / 741  = z
i =  195 / 741  = z
i =  196 / 741  = z
compute_simil >= 0.9 para [i: 196 ] =  artigo 137 da CLT  [j: 525 ] =  art. 137 da CLT
Tam. DF:  740
i =  197 / 740  = z
i =  198 / 740  = z
i =  199 / 740  = z
compute_simil >= 0.9 para [i: 199 ] =  Lei. 11. 941 / 09  [j: 372 ] =  Lei 11. 941 / 2009
Tam. DF:  739
i =  200 / 739  = z
i =  201 / 739  = z
compute_simil >= 0.9 para [i: 201 ] =  artigo 459, 8 1º, da CLT  [j: 468 ] =  art. 459, 8 único, CLT
Tam. DF:  738
compute_simil >= 0.9 para [i: 201 ] =  artigo 459, 8 1º, da CLT  [j: 590 ] =  artigo 457 8

compute_simil >= 0.9 para [i: 313 ] =  item V, da Súmula 331, do C. TST  [j: 585 ] =  Súmula 331, V, do C. TST
Tam. DF:  697
compute_simil >= 0.9 para [i: 313 ] =  item V, da Súmula 331, do C. TST  [j: 615 ] =  item V da Súmula 331 do TST
Tam. DF:  696
i =  314 / 696  = z
i =  315 / 696  = z
i =  316 / 696  = z
i =  317 / 696  = z
i =  318 / 696  = z
i =  319 / 696  = z
i =  320 / 696  = z
i =  321 / 696  = z
i =  322 / 696  = z
i =  323 / 696  = z
i =  324 / 696  = z
i =  325 / 696  = z
i =  326 / 696  = z
compute_simil >= 0.9 para [i: 326 ] =  art. 37, da CRFB  [j: 469 ] =  art. 37 da CRFB
Tam. DF:  695
i =  327 / 695  = z
i =  328 / 695  = z
i =  329 / 695  = z
i =  330 / 695  = z
i =  331 / 695  = z
compute_simil >= 0.9 para [i: 331 ] =  Súmula 331, VI [ 1 ], do C. TST  [j: 666 ] =  Súmula 331, VI1, do C. TST
Tam. DF:  694
i =  332 / 694  = z
i =  333 / 694  = z
i =  334 / 694  = z
compute_simil >= 0.9 para [i: 334 ] =  CDC, art. 104, primeira parte  [j: 367 ] =  CDC, art. 104, seg

compute_simil >= 0.9 para [i: 514 ] =  art. 459, parágrafo único, da CLT  [j: 623 ] =  CLT, art. 459, parágrafo único
Tam. DF:  662
i =  515 / 662  = z
i =  516 / 662  = z
i =  517 / 662  = z
i =  518 / 662  = z
i =  519 / 662  = z
i =  520 / 662  = z
i =  521 / 662  = z
i =  522 / 662  = z
i =  523 / 662  = z
i =  524 / 662  = z
i =  525 / 662  = z
i =  526 / 662  = z
i =  527 / 662  = z
i =  528 / 662  = z
i =  529 / 662  = z
i =  530 / 662  = z
i =  531 / 662  = z
i =  532 / 662  = z
compute_simil >= 0.9 para [i: 532 ] =  taxa Selic  [j: 593 ] =  Selic
Tam. DF:  661
i =  533 / 661  = z
i =  534 / 661  = z
i =  535 / 661  = z
i =  536 / 661  = z
i =  537 / 661  = z
i =  538 / 661  = z
i =  539 / 661  = z
i =  540 / 661  = z
i =  541 / 661  = z
i =  542 / 661  = z
i =  543 / 661  = z
i =  544 / 661  = z
i =  545 / 661  = z
i =  546 / 661  = z
i =  547 / 661  = z
i =  548 / 661  = z
i =  549 / 661  = z
i =  550 / 661  = z
i =  551 / 661  = z
i =  552 / 661  = z
i =  553 / 661  = z
i = 

In [23]:
print(len(df_categ_fund_50))

811


In [ ]:
laço externo para percorrer as linhas do df e pra cada linha instanciar uma lista com a coluna entsFundamentacao
Para cada item desta lista, percorrer o Dicionario de Entidades padronizadas e quando houver correspondencia de
similaridade maior que 90%, armazenar esta Entidade_Padronizada (do dicionario de entidades) numa nova lista 
temporaria.
Apos percorrer todo dicionario para todos os itens desta lista de Entidades, substituir a lista no DF original
pela lista temporaria criada

In [ ]:
Depois disso, montar um DF de contingencia, contando a frequencia de cada token em cada classificacao
        Procedente | Improcedente | Parcialmente Procedente | Acordo | Freq_Total
Token_1
Token_2
Token_3 
 ...
Token_801

Aplicar DF_Contingencia.corr(Pearson)?
Fazer graficos?

In [ ]:
# Apos concluir este trecho, voltar a fazer a analise dos tokens 50% -> 80% (+/- 1000 linhas) e repetir a execucao
df_categ_fund_50 = df_categ_fund_50 + df_categ_fund[1125:]   # <===
df_categ_fund_50


In [ ]:
# Falso positivos com: artigo 467 da CLT
# artigo 464 da CLT
# art. 461 da CLT
# Qual é o menor indice retornado pela funcao de similaridade que permite distinguir estas duas situacoes acima?
# Incluir no DF as duas situacoes acima, e ajustar o indice senão na substituicao do DF original serao periddas!
#
# Ir testando variacaoes das duas descricoes ate encontrar termos que o modelo de similaridade (funcao simil)
# obtenha um score/indice menor que 0.9, preservando sua semantica intrinseca. Incluir/substituir estes termos
# no DF_Dicionarios_De_Entidades_Da_Fundamentacao antes de aplica-lo ao DF_original.

In [ ]:
# Falso positivos com: artigos 186 e 927 do Código Civil
# artigos 186, 187 e 927 do Cód
# artigos 186 e 927, caput, do Código Civil
# artigos 186 e seguintes e 927 e seguintes do Código Civil

In [ ]:
# Falso positivos com: artigos 467 e 477 da CLT
# artigo 477 da CLT
# artigo 477, 88º, da CLT
# artigo 477, 48º, da CLT

In [ ]:
# Falso positivos com: artigo 883 da CLT 
# art. 880 da CLT
# art. 889 da CLT

In [ ]:
# Falso positivos com: art. 71, 8 1º, da Lei nº 8. 666 / 93
# artigo 71 da Lei 8. 666 / 93
# artigo 71, 8 1º, da Lei nº 8. 666 / 983   inserir: artigo 71,  §1º, da Lei nº 8.666/93
# substituir art. 71, 8 1º, da Lei nº 8. 666 / 93 por: artigo 71 da Lei 8.666/93
# Qual indice permite distinguir 'artigo 71 da Lei 8.666/93' de 'artigo 71,  §1º, da Lei nº 8.666/93'

In [ ]:
#Alterar Lei 8. 666 / 93 por Lei 8.666/93

In [ ]:
# Falso positivos com: artigo 37 da Constituição Federal
# artigo 37  §6º da Constituição Federal

In [ ]:
# Falso positivos com: artigo 455 da CLT
# artigo 459 da CLT
# artigo 457 da CLT
# Substituir artigo 459, 81º, da CLT por: artigo 459, §1º, da CLT

In [ ]:
# Falso positivos com: Súmula 331 do TST
# Súmula 338 do TST
# Súmula 329 do C. TST

In [ ]:
# Falso positivos com: Súmula 368 do TST
# Súmula 362 do TST
# Súmula 363 do TST

In [ ]:
# Falso positivos com: artigo 791 - A da CLT
# artigo 790 da CLT
# artigo 790, §3º da CLT

In [ ]:
# Falso positivos com: artigo 71 da Lei 8666 / 93
# parágrafo primeiro do artigo 71 da Lei 8. 666 / 93


In [ ]:
# Falso positivos com: art. 818 da CLT
# artigos 818 da CLT
# Artigo 818 da CLT

In [ ]:
# Falso positivos com: art. 477, 8 8º, da CLT
# art. 477, 8 8º, e 467 da CLT
# arts. 459, 8 1º e 477, 8 6º, da CLT
# Substituir 'art. 477, 8 8º, da CLT' por 'artigo 477, § 8º, da CLT'

In [ ]:
# Falso positivos com: art. 406 do Código Civil
# art. 402, do Código Civil


In [ ]:
# Falso positivos com: artigo 402 do Código Civil
# artigo 404 do Código Civil

In [ ]:
# Falso positivos com: CLT, art. 818 c / c do CPC / 15, art. 373, inciso |
# CLT, art. 818 c / c do CPC / 15, art. 378, inciso |

In [ ]:
# Substituir '$ 8º do artigo 477 da CLT' por 'artigo 477, parágrafo 8º, da CLT'

In [ ]:
# Falso positivos com: art. 844 da CLT
# art. 840 da CLT

In [ ]:
# Falso positivos com: Lei nº 5. 584 / 70
#  Lei nº 5. 645 / 70

In [ ]:
# Falso positivos com:  art. 2º da CLT
#   art. 8º, CLT
#   artigo 2º da CLT
#   art. 9º da CLT
#   artigo 9º da CLT
#   artigos 2º e 3º da CLT
#   artigo 3º da CLT

In [ ]:
# Falso positivos com:  art. 769 da CLT
#  art. 765 da CLT
#  

In [ ]:
# Duvida: OJ 363 da SDI - 1 do TST    OJ 363 da SBDI - 1 do TST

In [ ]:
# Substituir 'art. 790, 8 3º, da CLT' por 'art. 790, § 3º, da CLT'

In [ ]:
# Falso positivos com:  art. 896, CLT
#   art. 899, 84º, da CLT

In [ ]:
# Falso positivos com: Súmula 331, IV, do TST
#  Súmula 331, incisos IV e V, do C. TST

In [ ]:
# Ver correspondencia entre ADC 16 e Ação Declaratória de Constitucionalidade nº 16
# Ação Declaratória de Constitucionalidade nº 16  e   Ação Direta de Constitucionalidade nº 16

In [ ]:
# Analisar Lei nº 8. 666 / 98 ou 93?
# Lei nº 8. 666 / 98  e  Lei nº 8. 660 / 98

In [ ]:
# Alterar: Súmula nº 331, V, desta Corte  para  Súmula nº 331, V

In [ ]:
# Alterar 'parágrafo primeiro do artigo 71 da Lei 8. 666 / 98'  por 'artigo 71 da Lei 8.666/98'

In [ ]:
# Substituir 'artigos 58, Il e 67, ambos da Lei 8666 / 98'  por 'artigos 58, II e 67, ambos da Lei 8.666/98'

In [ ]:
# Falso positivos com: Súmula 461 do TST
#   Súmula 463 do TST

In [ ]:
# art. 1º - F da Lei nº 9. 494  [j: 89 ] =  artigo 1º - F da Lei nº 9. 494 / 97

In [ ]:
# Falso positivos com: Súmula 305 do TST
#  Súmula 308 do C. TST

In [ ]:
# Alterar: artigo 3º, III, da CF

In [ ]:
# Avaliar   OJ 400 da SDI - 1 do C. TST  [j: 196 ] =  OJ 400 da SBDI - 1 do TST

In [ ]:
# Falso positivos com: artigo 3º, III, da CF    [i: 88 ]
#  artigo 1º, III, da CF

In [ ]:
# Falso positivos com: art. 832, 83º da CLT
# parágrafo 3º do artigo 832 da CLT
# art. 832, 83º da CLT  [j: 417 ] =  art. 830, CLT
# Substituir 'art. 832, 83º da CLT' por 'artigo 832, §3º da CLT'

In [ ]:
# Falso positivos com: [i: 93 ] =  0482  e  [j: 636 ] =  0481

In [ ]:
# Falso positivos com: [i: 94 ] =  artigo 487, 8 3º, da CLT  [j: 287 ] =  art. 487, 8 1º, CLT
# Substituir 'artigo 487, 8 3º, da CLT' por 'artigo 487, § 3º, da CLT'

In [ ]:
# Substituir Lei nº 10. 035   por [j: 541 ] =  Lei 10. 035 / 00

In [ ]:
# Substituir 'art. 373, Il, do CPC de 2015' por 'artigo 373, II, do CPC'
#  [j: 230 ] =  art. 373, Il, do CPC

In [ ]:
# Falso positivos com: Consolidação das Leis do Trabalho  [j: 474 ] =  artigo 9º da Consolidação das Leis do Trabalho

In [ ]:
# Falso positivos com: 87º do art. 879 da CLT  [j: 743 ] =  artigo 879, caput, da CLT
# Substituir por 'artigo 879 da CLT'

In [ ]:
# Substituir art. 832, 8 3º, da CLT por art. 832 da CLT

In [ ]:
# Falso positivos com: artigo 114, VIII, da Constituição Federal  [j: 616 ] =  artigo 114, inciso VII, da Constituição Federal

In [ ]:
# Substituir súmula 368, item Il do C. TST por súmula 368, item II, do TST

In [ ]:
# Falso positivos com: artigo 344 do CPC  [j: 531 ] =  art. 341 do CPC

In [ ]:
# Falso positivos com: aris. 58, Ill, e 67 da Lei 8. 666 / 93
#  arts. 58, Ill, e 67 da Lei nº 8. 666 / 93
#  Substituir aris. 58, Ill, e 67 da Lei 8. 666 / 93 por arts. 58, III, e 67 da Lei 8.666/93

In [ ]:
# Parei em i = 150

In [31]:
df_categ_fund_50.to_csv(dir+"df_categ_fund_50.csv", index=False)

In [ ]:
# Tratando os 1125 primeiros itens (mais frequentes e que representam 50% do total de ocorrencias no texto)
z = len(df_categ_fund_50) # Podemos segmentar o DF df_categ_fund[0:1125] em df_categ_fund_50 e colocar o j ate len(df_categ_fund_50)

#for i in tqdm(range(z-1)):
i = 0
while i < (z-1):
    print("i = ",i, "/",z, " = z")
    #for j in range(i+1,z):
    j = i+1
    while j < z:   # Nao pode ser <= pois o array começa em 0.
        print("j = ",j)
        if compute_simil(df_categ_fund_50['ents'][i],df_categ_fund_50['ents'][j]) >= 0.9:
            print("compute_simil >= 0.9 para [i:",i,"] = ",df_categ_fund_50['ents'][i]," [j:",j,"] = ",df_categ_fund_50['ents'][j])
            df_categ_fund_50['freq'][i] = df_categ_fund_50['freq'][i] + df_categ_fund_50['freq'][j] # Acumula a frequencia
            #df_categ_fund['ents'][j] = df_categ_fund['ents'][i] # Ou podemos excluir as demais/seguintes entidade equivalentes
            df_categ_fund_50.drop(df_categ_fund_50.index[j], inplace=True) # Exclui o item duplicado
            df_categ_fund_50.reset_index(drop=True, inplace=True) # Atualiza o indice do DF.
            z = len(df_categ_fund_50)
            print("Tam. DF: ", z)
        j = j + 1
    i = i + 1

df_categ_fund_50.sort_values(by=['freq'], ascending=False, ignore_index=True, inplace=True)

In [ ]:
# Apos as alteracoes acima
# Refazer o calculo da metade da frequencia acumulada
#
# Podemos usar TF/IDF nas listas de tokens de fundamentacao?
# Neste indice, quais sao os termos mais importantes? Todos estes ja estao no Dicionario de Entidades?

In [ ]:
# Dois lotes de operacoes:
# 1) Processamento do Dicionario (DF) de Entidades
# 2) Atualizacao do DF original com o Dicionario de Entidades para padronizacao da correlação
# 3) Refazer a extração de entidades para dicionario e calculo da metade da frequencia acumulada

In [ ]:
# Threshold para similaridade semantica estabelecido em 0.9(90%) pela função compute_simil()
# Acima deste resultado, considerar que se trata da mesma entidade, pode-se então substituir/padronizar!

In [ ]:
%%time



z = 100

# Usar tqdm no for do i: df.index
#for i in tqdm(range(len(df_categ_fund)-1), desc = 'Barra de Progresso:'):
for i in tqdm(range(0,z-1), desc = 'Barra de progresso:'):     #len(df_categ_fund)-1), desc = 'Barra de Progresso:'):
    print("i = ",i)
    for j in range(i+1,z)):
        print("j = ",j)
        if compute_simil(df_categ_fund['ents'][i],df_categ_fund['ents'][j]) >= 0.9:
            print("compute_simil >= 0.9 para i:",i," j:",j)
            df_categ_fund['ents'][j] = df_categ_fund['ents'][i] # Ou podemos atribuir a uma lista de Entidades
      

In [ ]:
len(df_categ_fund)

In [ ]:
%%time
# Cria um novo DF contendo as Entidades padronizadas e prepara uma segunda coluna para receber as frequencias.
list_entsUnique = list(set(df_categ_fund['ents']))

# Cria um df_entsProcedente para calcular a frequencia das ents do set(que é uma list) acima.
list_zero = list()
list_zero = [0]*len(list_entsUnique)

df_entsUnique = pd.DataFrame(list(zip(list_entsUnique, list_zero)), columns =['ents','freq'])

In [ ]:
df_entsUnique 

In [ ]:
%%time
# Conta a frequencia sempre que a correspondencia for maior que 0.9:
# Percorre cada item da lista de cada uma das linhas e verifica se sua similaridade com algum item do df_entsUnique é maior que 0.9 e neste caso computa a frequencia.
#for i in tqdm(range(len(df_categ_fund)-1), desc = 'Barra de Progresso:'):
for i in tqdm(range(len(df)), desc = 'Barra de Progresso:'): # Em i percorre-se as linhas do DF.
    print("i = ", i)
    listEnt = df['entsFundamentacao'][i]  # Atribui a lista dessa linha a listEnt
    
    for j in tqdm(range(len(listEnt))):    # Em j percorre-se os itens da lista da linha
        print("j = ", j)
        for k in range(len(df_entsUnique)): # Em k percorre-se as Entidades unicas e calcula a frequencia baseada na similiaridade do cosseno dos embeddings do modelo de similaridade semantica.
            print("k = ", k)
            if compute_simil(df['entsFundamentacao'][i][j], df_entsUnique['ents'][k]) >= 0.9:
                print("compute_simil >= 0.9 em i:",i," j:",j," k:",k)
                #pos = df_entsUnique.index[(df_entsUnique['ents'] == df['entsFundamentacao'][i][j])].tolist()[0]
                df_entsUnique['freq'][k] = int(df_entsUnique['freq'][k]) + 1
                df['entsFundamentacao'][i][j] = df_entsUnique['ents'][k]  # Padroniza as Entidades no Dataframe DF para calculo posterior da correlação

df_entsUnique.sort_values(by=['freq'], ascending=False, ignore_index=True, inplace=True)

In [ ]:
df_entsUnique['freq'][0] 

In [ ]:
df 

In [ ]:
df_entsUnique 

In [ ]:
%%time
# DF agora esta preparadado para aplicacao da Correlação de Pearson nas Entidades Padronizadas (ver notebook)
# https://datascience.stackexchange.com/questions/893/how-to-get-correlation-between-two-categorical-variable-and-a-categorical-variab
# https://stackoverflow.com/questions/48035381/correlation-among-multiple-categorical-variables-pandas
#
for i in tqdm(range(len(df)), desc = 'Barra de Progresso:'): # Em i percorre-se as linhas do DF.
    listEnt = df['entsFundamentacao'][i]  # Atribui a lista dessa linha a listEnt
    for j in range(len(listEnt)):    # Em j percorre-se os itens da lista da linha
      for k in range(len(df_entsUnique)): # Em k percorre-se as Entidades unicas e calcula a frequencia baseada na similiaridade do cosseno dos embeddings do modelo de similaridade semantica.
        if compute_simil(df['entsFundamentacao'][i][j], df_entsUnique['ents'][k]) > 0.9:
            if df['classificacao'][i][j] == "Procedente":
              df_entsUnique['Proc'][k] = int(df_entsUnique['Proc'][k]) + 1

            if df['classificacao'][i][j] == "Improcedente":
              df_entsUnique['Improc'][k] = int(df_entsUnique['Improc'][k]) + 1
              
            if df['classificacao'][i][j] == "Parcialmente procedente":
              df_entsUnique['ParcProc'][k] = int(df_entsUnique['ParcProc'][k]) + 1
              
            if df['classificacao'][i][j] == "Acordo ou outros":
              df_entsUnique['Proc'][k] = int(df_entsUnique['Proc'][k]) + 1




In [ ]:
%%time
# Salva o Pandas DataFrame
df_entsUnique.to_csv(dir+"cont_table_data.csv", index=False)
df.to_csv(dir+"raw_data.csv", index=False)


In [ ]:
# Let's perform a Chi-Squred test. Null hypothesis: they are independent, Alternative hypothesis is that they are correlated in some way.
# # So we run the chi-squared test and the resulting p-value here can be seen as a measure of correlation between these two variables.
#
# To compute Crammer's V we first find the normalizing factor chi-squared-max which is typically the size of the sample, divide the chi-square by it and take a square rootUnder the Null hypothesis, we assume uniform distribution.
# 
# # Uma vez salvo o DF da tabela de contingencia em .csv podemos carregá-lo e exectuar os comandos em R para teste de Chi2 e Crammer 's V.
#

In [ ]:
# Quais os principais Temas/Assuntos de ACP trabalhista?
# https://www.cecult.ifch.unicamp.br/pf-cecult/public-files/projetos/10596/temario_completo_do_ministerio_publico_do_trabalho.pdf

In [ ]:
1º  2º   simil       compare 
0   2    0.9753035   0.6029748160380571
2   54   0.9919835   1.0
54  146  0.91689515  1.0


In [ ]:
df_categ_fund['ents'][0]

In [ ]:
df_categ_fund['ents'][2]

In [ ]:
df_categ_fund['ents'][54]

In [ ]:
df_categ_fund['ents'][146]

In [ ]:
s1 = "art. 467, da CLT"
s2 = "art. 67, da CLT"

In [ ]:
print("Similaridade entre as sentenças: ", compute_simil(s1,s2))

In [ ]:
compare(s1, s2)

In [ ]:
# Threshold para similaridade semantica estabelecido em 0.9(90%) pela função compute_simil()
# Acima deste resultado, considerar que se trata da mesma entidade, pode-se então substituir/padronizar!

In [ ]:
len(df_categ_fund)


In [ ]:
df_test.reset_index(drop=True, inplace=True)

In [ ]:
df_categ_fund[0:50]

In [ ]:
df_categ_fund[51:100]

In [ ]:
df_categ_fund[101:150]

In [ ]:
df_categ_fund[151:200]

In [ ]:
df_categ_fund[201:250]

In [ ]:
df_categ_fund[251:300]

In [ ]:
df_categ_fund[301:350]

In [ ]:
df_categ_fund[351:400]

In [ ]:
df_categ_fund[401:450]

In [ ]:
df_categ_fund[451:500]

In [ ]:
df_categ_fund[501:550]

In [ ]:
df_categ_fund[551:600]

In [ ]:
df_categ_fund[951:1000]

In [ ]:
df_categ_fund[201:250]

In [ ]:
df_categ_fund[201:250]

In [ ]:
df_categ_fund[201:250]

In [ ]:
df_categ_fund[201:250]